In [ ]:
# Choose variant and machine type
VARIANT = '2b-it' #@param ['2b', '2b-it', '9b', '9b-it', '27b', '27b-it']
MACHINE_TYPE = 'cuda' #@param ['cuda', 'cpu']
PRINT_INFO = False #@param bool

CONFIG = VARIANT[:2]
if CONFIG == '2b':
  CONFIG = '2b-v2'

In [ ]:
# Choose seed and PAWS parameters
SEED = 42 # @param int the answer to life, the universe, and everything

USE_PAWS = True #@param bool
PAWS_PERCENTILE = 25 # @param int

paws_dict_path = "paws_dict/gsm8k_paws_dict.json"# @param str

In [ ]:
import os
from google.colab import userdata # `userdata` is a Colab API.

os.environ["KAGGLE_USERNAME"] = 'ucheochuba'
os.environ["KAGGLE_KEY"] = '5f5dd974f13fe89c391f02b0cf74e892'

In [ ]:
!pip install -q -U torch immutabledict sentencepiece

In [ ]:
!pip install datasets
!pip install evaluate

In [ ]:
import torch
import os
import kagglehub
from datasets import load_dataset
from evaluate import load  # Use evaluate instead of load_metric


In [ ]:
# Download weights directory from Kaggle
try:
    weights_dir = kagglehub.model_download(f'google/gemma-2/pyTorch/gemma-2-{VARIANT}')
    print(f"Downloaded weights to: {weights_dir}")
except Exception as e:
    raise RuntimeError(f"Failed to download model weights: {e}")

Downloaded weights to: /kaggle/input/gemma-2/pytorch/gemma-2-2b-it/1


In [ ]:
 # clone in PAWS threshold dictionary
# Initialize a new git repository
!git init

# Add the remote repository
!git remote add origin https://github.com/ucheochuba/cs229s-final.git

# Enable sparse-checkout
!git config core.sparseCheckout true

# Set the folder path you want to checkout (e.g., paws_dict)
!echo "paws_dict/*" >> .git/info/sparse-checkout

# Pull the specific folder from the repository
!git pull origin main

Reinitialized existing Git repository in /content/.git/
error: remote origin already exists.
From https://github.com/ucheochuba/cs229s-final
 * branch            main       -> FETCH_HEAD
Already up to date.


In [ ]:
# NOTE: The "installation" is just cloning the repo.
# !git clone https://github.com/google/gemma_pytorch.git
!git clone https://github.com/ucheochuba/paws_gemma_pytorch gemma_pytorch

fatal: destination path 'gemma_pytorch' already exists and is not an empty directory.


In [ ]:
import sys
sys.path.append('gemma_pytorch')
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/usr/local/lib/python3.10/dist-packages/setuptools/_vendor', '/root/.ipython', '/tmp/tmp1715yo2i', 'gemma_pytorch', 'gemma_pytorch']


In [ ]:
from gemma.model import GemmaForCausalLM
from gemma.tokenizer import Tokenizer
from gemma.config import get_model_config
import kagglehub

In [ ]:
# Verify the presence of tokenizer and checkpoint files
tokenizer_path = os.path.join(weights_dir, 'tokenizer.model')
if not os.path.isfile(tokenizer_path):
    raise FileNotFoundError(f"Tokenizer not found at: {tokenizer_path}")

ckpt_path = os.path.join(weights_dir, 'model.ckpt')
if not os.path.isfile(ckpt_path):
    raise FileNotFoundError(f"PyTorch checkpoint not found at: {ckpt_path}")

print("Tokenizer and checkpoint files verified.")

Tokenizer and checkpoint files verified.


### PAWS Stuff

In [ ]:
# load in files
import json

with open(paws_dict_path, 'r') as f:
    paws_dict = json.load(f)

In [ ]:
# Function to convert keys to integers recursively
def convert_keys_to_int(d):
    if isinstance(d, dict):
        # Create a new dictionary with integer keys
        return {int(k) if k.isdigit() else k: convert_keys_to_int(v) for k, v in d.items()}
    elif isinstance(d, list):
        # If the value is a list, recursively process each element
        return [convert_keys_to_int(item) for item in d]
    else:
        return d

# convert keys from str to int
paws_dict = convert_keys_to_int(paws_dict)

In [ ]:
dataset = load_dataset("gsm8k", "main", cache_dir='/tmp')
dataset = dataset.shuffle(seed=42)

# Load Gemma 2B model
MODEL_VARIANT = "2b-v2"  # Update if needed
model_config = get_model_config(MODEL_VARIANT)
model_config.tokenizer = tokenizer_path
model_config.quant = 'quant' in VARIANT

# Load tokenizer and model
# tokenizer = Tokenizer(tokenizer_path)
torch.set_default_dtype(model_config.get_dtype())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GemmaForCausalLM(model_config, use_paws=USE_PAWS, paws_percentile=PAWS_PERCENTILE, paws_dict=paws_dict)
model.load_weights(ckpt_path)
model.to(device).eval()

Initializing attention layer 0 with PAWS activated. Cutoff percentile: 25.
Initializing attention layer 1 with PAWS activated. Cutoff percentile: 25.
Initializing attention layer 2 with PAWS activated. Cutoff percentile: 25.
Initializing attention layer 3 with PAWS activated. Cutoff percentile: 25.
Initializing attention layer 4 with PAWS activated. Cutoff percentile: 25.
Initializing attention layer 5 with PAWS activated. Cutoff percentile: 25.
Initializing attention layer 6 with PAWS activated. Cutoff percentile: 25.
Initializing attention layer 7 with PAWS activated. Cutoff percentile: 25.
Initializing attention layer 8 with PAWS activated. Cutoff percentile: 25.
Initializing attention layer 9 with PAWS activated. Cutoff percentile: 25.
Initializing attention layer 10 with PAWS activated. Cutoff percentile: 25.
Initializing attention layer 11 with PAWS activated. Cutoff percentile: 25.
Initializing attention layer 12 with PAWS activated. Cutoff percentile: 25.
Initializing attention

GemmaForCausalLM(
  (embedder): Embedding()
  (model): GemmaModel(
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): GemmaAttention(
          (qkv_proj): Linear()
          (o_proj): Linear()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear()
          (up_proj): Linear()
          (down_proj): Linear()
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
        (pre_feedforward_layernorm): RMSNorm()
        (post_feedforward_layernorm): RMSNorm()
      )
    )
    (norm): RMSNorm()
  )
  (sampler): Sampler()
)

In [ ]:
import random
from tqdm import tqdm
# from compute_score import *

In [ ]:
# @title GSM8K Prompts

PREAMBLE = """As an expert problem solver solve step by step the following mathematical questions."""

# The default gsm8k prompt from the CoT paper
# https://arxiv.org/pdf/2201.11903.pdf page 35.

PROMPT = """Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted. So, they must have planted 21 - 15 = 6 trees. The answer is 6.

Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.

Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Leah had 32 chocolates and Leah's sister had 42. That means there were originally 32 + 42 = 74 chocolates. 35 have been eaten. So in total they still have 74 - 35 = 39 chocolates. The answer is 39.

Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of lollipops he has given to Denny must have been 20 - 12 = 8 lollipops. The answer is 8.

Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: He has 5 toys. He got 2 from mom, so after that he has 5 + 2 = 7 toys. Then he got 2 more from dad, so in total he has 7 + 2 = 9 toys. The answer is 9.

Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: There are 4 days from monday to thursday. 5 computers were added each day. That means in total 4 * 5 = 20 computers were added. There were 9 computers in the beginning, so now there are 9 + 20 = 29 computers. The answer is 29.

Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
A: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.

Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: She bought 5 bagels for $3 each. This means she spent 5 * $3 = $15 on the bagels. She had $23 in beginning, so now she has $23 - $15 = $8. The answer is 8."""


# Extension of the default 8-shot prompt, page 35 in
# https://arxiv.org/pdf/2201.11903.pdf
# The extension is intended to improve performance on
# more complicated gsm8k examples.

EXTRA_3_SHOTS = """As an expert problem solver solve step by step the following mathematical questions.

Q: Tina makes $18.00 an hour.  If she works more than 8 hours per shift, she is eligible for overtime, which is paid by your hourly wage + 1/2 your hourly wage.  If she works 10 hours every day for 5 days, how much money does she make?
A: Here's how to calculate Tina's earnings:

**Regular Time:**
- Hours per shift: 8 hours
- Wage per hour: $18.00
- Regular pay per shift: 8 hours * $18.00/hour = $144.00

**Overtime:**
- Overtime hours per shift: 10 hours - 8 hours = 2 hours
- Overtime pay per hour: $18.00 + ($18.00 / 2) = $27.00
- Overtime pay per shift: 2 hours * $27.00/hour = $54.00

**Total per day:**
- Regular pay + overtime pay: $144.00/shift + $54.00/shift = $198.00/day

**Total for 5 days:**
- 5 days * $198.00/day = $990.00

**Therefore, Tina will make $990.00 in 5 days.** The answer is 990.

Q: Abigail is trying a new recipe for a cold drink. It uses 1/4 of a cup of iced tea and 1 and 1/4 of a cup of lemonade to make one drink. If she fills a pitcher with 18 total cups of this drink, how many cups of lemonade are in the pitcher?
A: ## Ambiguity in the Problem Statement:

There is one main ambiguity in the problem statement:

**Total volume vs. Number of servings:** The statement "18 total cups of this drink" could be interpreted in two ways:
  * **18 cups of the combined volume:** This would mean Abigail used a total of 18 cups of liquid, including both iced tea and lemonade.
  * **18 individual servings:** This would mean Abigail made 18 individual drinks, each containing 1/4 cup of iced tea and 1 1/4 cup of lemonade.

Let us assume the interpretation "18 cups of the combined volume".

## Solution assuming 18 cups of combined volume:

**Step 1: Find the proportion of lemonade in one drink:**

* Lemonade: 1 1/4 cups
* Iced tea: 1/4 cup
* Total: 1 1/4 + 1/4 = 1 1/2 cups
* Lemonade proportion: (1 1/4) / (1 1/2) = 5/6

**Step 2: Calculate the amount of lemonade in the pitcher:**

* Total volume: 18 cups
* Lemonade proportion: 5/6
* Volume of lemonade: 18 * (5/6) = 15 cups

Therefore, there are 15 cups of lemonade in the pitcher. The answer is 15.

Q: A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?
A: Let us solve it using algebra. Let x be the number of people on the ship the monster ate in the first hundred years.

The number of people on the ship eaten in the second hundred years is 2x, and in the third hundred years is 4x.

Therefore, the total number of people eaten over three hundred years is x + 2x + 4x = 847.

Combining like terms, we get 7x = 847.

Dividing both sides by 7, we find x = 121.

Therefore, there were 121 people on the ship the monster ate in the first hundred years. The answer is 121."""

In [ ]:
import random
from tqdm import tqdm
import json
import re

TEMPLATE = """
Q: {question}
A:"""

# Helper function to format GSM8K prompts
# def format_gsm8k_prompt(question):
#     """
#     Format the question as a prompt for the Gemma model.
#     """
#     return f"<start_of_turn>user\n{question}<end_of_turn><eos>\n<start_of_turn>model\n"

# Generate model answers
def generate_answer(model, prompt, max_length=128, device="cuda"):
    """
    Generate an answer using the Gemma model.
    """
    outputs = model.generate(prompt, output_len=max_length, device=device)
    answer = outputs.split("<end_of_turn>")[0].split("<start_of_turn>model\n")[-1]
    return answer.strip()

# Evaluation function
def evaluate_gsm8k_subset(model, dataset, device="cuda", num_samples=500, random_seed=42, print_info=False):
    """
    Evaluate the Gemma model on a subset of GSM8K and compute accuracy.
    """
    random.seed(random_seed)
    references = []

    # Randomly select `num_samples` examples
    test_dataset = dataset["test"]
    sampled_indices = random.sample(range(len(test_dataset)), min(num_samples, len(test_dataset)))
    sampled_data = [test_dataset[idx] for idx in sampled_indices]

    responses = {}
    idx = 0
    correct = 0

    for task_id, problem in tqdm(enumerate(sampled_data), desc="Evaluating GSM8K"):
      if task_id in responses: continue

      # Formulate and print the full prompt
      full_prompt = (PREAMBLE +'\n\n' + PROMPT + '\n' +
                    TEMPLATE.format(question=problem['question']))

      response = generate_answer(model, full_prompt, max_length=128, device=device)
      responses[task_id] = response.split('Q:')[0]

      if task_id % 10 == 0 and print_info:
        print(f"\nPrompt: {problem['question']}")
        print(f"Answer: {find_number(problem['answer'])}")
        print(f"Response: {find_number(responses[task_id])}")

      correct_answer = False
      try:
        correct_answer = float(maybe_remove_comma(
            find_number(problem['answer']))) == float(responses[task_id])
      except:
        correct_answer = maybe_remove_comma(
            find_number(problem['answer'])) == maybe_remove_comma(
                find_number(responses[task_id]))

      if correct_answer:
        correct += 1

      result = {
          "question": problem['question'],
          "answer": problem['answer'],
          "model_prediction": response,
          "correct": correct_answer
      }
      # print(json.dumps(result, indent=4))

      references.append({
          "id": task_id,
          "result": result
      })


      if idx % 10 == 0 and print_info:
        print('-'*40)
        print(f"Correct: {correct} out of {idx+1}")
        print("="*40)
        # print(f"Accuracy: {correct/num_samples}")

      idx += 1

    path = f'percentile_{PAWS_PERCENTILE}_gsm8k_evaluation_results.json'
    try:
        with open(path, 'w') as f:
            json.dump(references, f, indent=4)
        print(f"Evaluation results saved to {path}")
    except Exception as e:
        print(f"Error saving results: {e}")

    print(f"Accuracy: {correct/num_samples}")
    # return responses

# Utilities
def find_numbers(x: str) -> list[str]:
  """Finds all numbers in a string."""
  # Search for number, possibly negative (hyphen), with thousand separators
  # (comma), and with a decimal point (period inbetween digits).
  numbers = re.compile(
      r'-?[\d,]*\.?\d+',
      re.MULTILINE | re.DOTALL | re.IGNORECASE,
  ).findall(x)
  return numbers


def find_number(x: str,
                answer_delimiter: str = 'The answer is') -> str:
  """Finds the most relevant number in a string."""
  # If model uses the answer delimiter, then select the first number following
  # that format.
  if answer_delimiter in x:
    answer = x.split(answer_delimiter)[-1]
    numbers = find_numbers(answer)
    if numbers:
      return numbers[0]

  # In general, select the last number in the string.
  numbers = find_numbers(x)
  if numbers:
    return numbers[-1]
  return ''


def maybe_remove_comma(x: str) -> str:
  # Example: 5,600 -> 5600
  return x.replace(',', '')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Run evaluation
evaluate_gsm8k_subset(model, dataset, num_samples=500, print_info=PRINT_INFO)

Evaluating GSM8K: 500it [56:42,  6.80s/it]

Evaluation results saved to percentile_25_gsm8k_evaluation_results.json
Accuracy: 0.014


In [ ]:
# retrieve json from disk
from google.colab import files
files.download(f'percentile_{PAWS_PERCENTILE}_gsm8k_evaluation_results.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>